## SDP application for BB84 protocol

We must first call the appropriate functions to implement the problem.

We have also defined an "operator" class to help us manipulate the fixtures in the big matrix T

The "reduce", "arrange" and "ReduceList" functions are use to simplifies the expression of the multiplication of multiple operators

The "NPAhierarchy" function builds the entire hierarchy at the level considered



In [1]:
import numpy as np
import picos
import cvxpy as cp
from scipy.stats import norm
import math
import sys
from pprint import pprint
from matplotlib import pyplot as plt

In [4]:
class operator:
    
    def __init__(self,h,m,o):  
        
        self.m = m  # measurement basis  
        self.o = o  # measurement projector, here we always consider outcome 0
        self.h = h  # Hilbert space  0 for Bob 1 for Eve
        
        
    def __repr__(self):
        rep = []
        i =0
        while i<(len(self.h)):
            if self.h[i]==0:
                rep = rep + ['B_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==1:
                rep = rep + ['E_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==-1:
                rep = rep + ['Id']
            i = i+1
            
        return ''.join(rep)
    
    
    def __hash__(self):
        return hash((str(self.h), str(self.m), str(self.o))) 
    
                    
    def __mul__(self, other):
        return operator(self.h+other.h, self.m+other.m, self.o+other.o)
    
    
    __rmul__=__mul__
              
        
def reduce(OP):  # Simplifies the expression of the multiplication of multiple operators


    i=0
    l = len(OP.m)-1
    while i < l:
        ah = OP.h[i]
        ao = OP.o[i]
        am = OP.m[i]
        bh = OP.h[i+1]
        bo = OP.o[i+1]
        bm = OP.m[i+1]
        if ah != bh:
            if ah==1:  # Place the operator of Eve far right and those of Bob far left 
                OP.h[i], OP.h[i+1] = OP.h[i+1], OP.h[i]
                OP.o[i], OP.o[i+1] = OP.o[i+1], OP.o[i]
                OP.m[i], OP.m[i+1] = OP.m[i+1], OP.m[i]
            elif ah==-1:  # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i)
                (OP.o).pop(i)
                (OP.m).pop(i)
                l = l-1
                i = i-1
            elif bh==-1: # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
            else:
                pass
        elif ah==bh:
            if ah==1:  # 1 correspond to Eve
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
            elif ah==0:  # 0 correspond to Bob
                if am != bm:
                    pass
                else:
                    if ao==bo:
                        (OP.h).pop(i+1)
                        (OP.o).pop(i+1)
                        (OP.m).pop(i+1)
                        l = l-1
                        i = i-1   
                    else:
                        return 0
            elif ah==-1:
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
  
        i = i+1
    return OP
        
        
def arrange(OPToReduce):
    if len(OPToReduce.m)==1:
        pass
    else:
        for i in range(len(OPToReduce.m)):
            reduce(OPToReduce)
    return reduce(OPToReduce)



def Transp(OP):
    if OP == 0:
        return 0
    else:
        h = OP.h
        o = OP.o
        m = OP.m
        T = operator(h[::-1],m[::-1],o[::-1])
        Transp = arrange(T)
        return T
    

def ReduceList(Mon):
    for i,m in enumerate(Mon):
        for j,n in enumerate(Mon):
            if i!=j:
                if hash(m)==hash(n):
                    Mon.pop(j)
                else:
                    pass
            else:
                pass
        
    return Mon



In [5]:
# mesure x outcome a , mesure 
def NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,lvl):
    Max = NOutcomeB*NMesureB+NOutcomeE*NMesureE +1
    
    if lvl == 1:
        Mon = [operator([-1],[0],[0])]
        MonT = [operator([-1],[0],[0])]
        for k in range(NMesureB):
            for i in range(NOutcomeB):
                Mon = Mon + [operator([0],[k],[i])]
                MonT = MonT + [operator([0],[k],[i])]
                
        for k in range(NMesureE):
            for i in range(NOutcomeE):
                Mon = Mon + [operator([1],[k],[i])]
                MonT = MonT + [operator([1],[k],[i])]
            
    elif lvl == 1.5:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for e in range(NMesureE):
            for eo in range(NOutcomeE):
                for k in range(NMesureB):
                    for i in range(NOutcomeB):
                        Mon = Mon + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
                        MonT = MonT + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
        
    elif lvl == 2:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                #if i!=j:
                tot = arrange(n*m)
                totT = arrange(m*n)
                if tot !=0:
                    Mon = Mon + [tot]
                    MonT = MonT + [totT]
                else:
                    pass
                #else:
                 #   pass
    
    elif lvl == 3:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,2)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                for k,p in enumerate(Mon[1:Max]):
                    #if i!=j and i!=k and k!=j:
                    tot = arrange(arrange(m*n)*p)
                    totT = arrange(arrange(p*n)*m)
                    if tot != 0:
                        Mon = Mon + [tot]
                        MonT = MonT + [totT]
                    else:
                        pass
                    #else:
                        #pass
        
        
    else:
        pass
    
    Mon = ReduceList(Mon)
    MonT = ReduceList(MonT)
    return Mon, MonT

In [9]:
def BOUND(q):
    bound = 1/2 + np.sqrt(1-q**2)/2
    return bound


def fx(q):
    if np.sqrt(2)/2 <= q <= (1/2 + np.sqrt(2)/4):
        return 1 + 1/np.sqrt(2) - q
    elif (1/2 + np.sqrt(2)/4)< q <= 1:
        return 1/2 + np.sqrt(q*(1-q))
    elif q < np.sqrt(2)/2:
        return 1

Once those function are defined, we can construct a dictionary of the different element in the big matrix T.

This allows us to reduce the number of variables for the problem.

In [8]:
NMesureB = 2
NOutcomeB = 1
NMesureE = 1
NOutcomeE = 1
LVLNPA = 2
states = 4
print('NPA LVL{}'.format(LVLNPA))
Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,LVLNPA)

#Mon = ReduceList(Mon)
#MonT = ReduceList(MonT)
            
        

MAX = len(Mon)
print('Sub-block: {}x{}'.format(MAX,MAX))
print('Big matrix: {}x{}'.format(states*MAX,states*MAX))
print(Mon)
print(MonT)
terms = {}
#termsT = {}

for i,m in enumerate(Mon):
    for j,n in enumerate(MonT):
        tot = arrange(n*m)
        tot_rep = hash(tot)
        if tot_rep in terms:
            terms[tot_rep].append((j,i))
        else:
            terms[tot_rep] = [(j,i)]

print("Number of different elements in a sub-block :", len(terms.keys()))
ndif = len(terms.keys())*((states*(states+1))//2)
print("Number of different variables in the problem :",ndif)

NPA LVL2
Sub-block: 8x8
Big matrix: 32x32
[Id, B_(0|0), B_(1|0), E_(0|0), B_(1|0)B_(0|0), B_(0|0)E_(0|0), B_(0|0)B_(1|0), B_(1|0)E_(0|0)]
[Id, B_(0|0), B_(1|0), E_(0|0), B_(0|0)B_(1|0), B_(0|0)E_(0|0), B_(1|0)B_(0|0), B_(1|0)E_(0|0)]
Number of different elements in a sub-block : 16
Number of different variables in the problem : 160


Once the separate elements have been identified, the big matrix T is built, making sure that it is Hermitian

We implement then the Gram constraints and the statistical constraints

# PICOS

In [ ]:
z = picos.ComplexVariable("z", ndif)

Matlist = []
for X in range(4):
    for Y in range(X,4):
        for k in terms.keys():
            B = np.zeros((4*MAX,4*MAX))
            if k ==0:
                Matlist.append(B)
            else:
                for i in terms[k]:
                    B[X*MAX+i[0],Y*MAX+i[1]] = 1    
                Matlist.append(B)
                
            
if len(Matlist) != ndif:
    print("Mismatch parameter number, shoud be :",ndif, " insted of :",len(Matlist))
    print("end program")
    sys.exit(1)
    
print("Number parameter match")
print("Summing matrix")
T = sum([(Matlist[i]*z[i] + Matlist[i].T*z.conj[i])for i in range(ndif)])
print("cheking hermitian")
if T.hermitian==True:
    print('∑[v_i * B_i + conj(v_i)* Transp(B_i)] is hermitian')
else:
    print("T is not hermitian!!")
        
# Define the problem.
PB = picos.Problem()
#PA = picos.Problem()
SDPB = PB.add_constraint( T >> 0)
#SDPA = PA.add_constraint( T >> 0)

#Gram Matrix
Gram = np.array([[1,0,1/np.sqrt(2),1/np.sqrt(2)],
                   [0,1,1/np.sqrt(2),-1/np.sqrt(2)],
                   [1/np.sqrt(2),1/np.sqrt(2),1,0],
                   [1/np.sqrt(2),-1/np.sqrt(2),0,1]])

constraints = []

for X in range(4):
    for Y in range( 4):
        constraints.append(T[MAX*X, MAX*Y] == Gram[X, Y])
                                
GramConstB = PB.add_list_of_constraints(constraints)
#GramConstA = PA.add_list_of_constraints(constraints)


guessBlP = []
guessAlP = []
HBElP = []
HAElP = []
HBAlP = []
KeyRP = []
KeyDP = []

#Eve Guessing probabilitie on bob 
#The key is generetated with state 0 and 1 send 

PGuessBB = (T[0,0]+T[8,8]+2*T[1,3]-T[1,1]-T[3,3]+2*T[9,11]-T[9,9]-T[11,11])/2

#Eve Guessing probabilitie on Alice

PGuessAA = (T[3,3]+T[8,8]-T[11,11])/2

PGuessB = (PGuessBB.real)
#PGuessA = (PGuessAA.real)

PB.set_objective("max", PGuessB)
#PA.set_objective("max", PGuessA)


RANGE = [x*0.1 for x in range(10*0, 8*1)][0:-1] + [x*0.01 for x in range(7*10, 10*10)]

for q in RANGE:
    print("q =", q)
    constraints2 = [                #pabc  a=outcome, b=state send, c=mesurment basis
    
        T[1,1] == (q*1 + (1-q)*1/2),      #p00z
        T[2,2] == (q*1/2 + (1-q)*1/2),    #p00x
        T[9,9] == (q*0 + (1-q)*1/2),      #p01z
        T[10,10] == (q*1/2 + (1-q)*1/2),    #p01x
        T[17,17] == (q*1/2 + (1-q)*1/2),    #p0+z
        T[18,18] == (q*1 + (1-q)*1/2),    #p0+x
        T[25,25] == (q*1/2 + (1-q)*1/2),  #p0-z
        T[26,26] == (q*0 + (1-q)*1/2)]    #p0-x

    #W = (2*T[1,1]-2*T[9,9]+2*T[18,18]-2*T[26,26])/2    #witness

    StatConstB = PB.add_list_of_constraints(constraints2)
    #StatConstA = PA.add_list_of_constraints(constraints2)
    
    #PA.options.solver = "mosek"
    PB.options.solver = "mosek"
    #print('Solving using mosek ...')
    #PA.solve()
    PB.solve()
    
    
    guessBlP.append(PGuessB.value)
    #guessAlP.append(PGuessA.value)
    
    #print('PGuess value :',PGuess.value)
    
    print("Value with MOSEK guessB =", PGuessB.value)
    #print("Value with MOSEK guessA =", PGuessA.value)
    
    HBAP = -((1+q)*math.log2((1+q)/2) + (1-q)*math.log2((1-q)/2))/2
    InfHPguessBP = -math.log2(PGuessB.value)
    #InfHPguessAP = -math.log2(PGuessA.value)
    KeyRateReversP = InfHPguessBP - HBAP
    #KeyRateDirectP = InfHPguessAP - HBAP
    if KeyRateReversP < 0:
        KeyRP.append(0)
    elif KeyRateReversP > 0:
        KeyRP.append(KeyRateReversP)
    #if KeyRateDirectP < 0:
    #    KeyDP.append(0)
    #elif KeyRateDirectP > 0:
    #   KeyDP.append(KeyRateDirectP)
    
    print("Key Rate Revers", KeyRateReversP)
    #print("Key Rate Direct", KeyRateDirectP)
    
    print("******************")
    PB.remove_constraint((2,))
    #PA.remove_constraint((2,))
    
print("DONE")

# CVXPY

In [ ]:
#NPA hierarchi LVL2  
Matlist = []
for X in range(4):
    for Y in range(X,4):
        for k in terms.keys():
            B = np.zeros((4*MAX,4*MAX))
            if k ==0:
                Matlist.append(B)
            else:
                for i in terms[k]:
                    B[X*MAX+i[0],Y*MAX+i[1]] = 1    
                Matlist.append(B)

if len(Matlist) != ndif:
    print("Mismatch parameter number, shoud be :",ndif, " insted of :",len(Matlist))
    print("end program")
    sys.exit(1)
    
print("Number parameter match")
print("Summing matrix")

z = cp.Variable(ndif,complex=True)
#T = cp.Variable((4*MAX,4*MAX),  hermitian=True)
T = sum([(Matlist[i]*z[i] + Matlist[i].T*cp.conj(z[i])) for i in range(ndif)])
print("cheking hermitian")
if T.is_hermitian()==True:
    print('∑[v_i * B_i + conj(v_i)* Transp(B_i)] is hermitian')
else:
    print("T is not hermitian!!")
    
constraints = [T >> 0]


#Gram Matrix
Gram = np.array([[1,0,1/np.sqrt(2),1/np.sqrt(2)],
                   [0,1,1/np.sqrt(2),-1/np.sqrt(2)],
                   [1/np.sqrt(2),1/np.sqrt(2),1,0],
                   [1/np.sqrt(2),-1/np.sqrt(2),0,1]])

#Operatorzs set={1,B_0z,B_0x,E_0,B_0z*E_0,B_0x*E_0,B_0z*B_0x,B_0x*B_0z} 
#bloc matrix 8x8, 4 states send ==> T matrix 32x32 
#state1 ==> 0
#state2 ==> 1
#state3 ==> +
#state4 ==> -

#T = cp.Variable((32,32),  symmetric=True)

#constraints implementation

#constraints = [T >> 0]  # T is PSD

for X in range(4):
    for Y in range(X, 4):
        constraints.append(T[MAX*X, MAX*Y] == Gram[X, Y])
        
    
constraints2 = []        
# known Bob probabilities ( q*idieal + (1-q)*blanc noise)            
q = cp.Parameter(nonneg=True)
constraints2.extend([                #pabc  a=outcome, b=state send, c=mesurment basis
    
    T[1,1] == (q*1 + (1-q)*1/2),      #p00z
    T[2,2] == (q*1/2 + (1-q)*1/2),    #p00x
    T[MAX+1,MAX+1] == (q*0 + (1-q)*1/2),      #p01z
    T[MAX+2,MAX+2] == (q*1/2 + (1-q)*1/2),    #p01x
    T[2*MAX+1,2*MAX+1] == (q*1/2 + (1-q)*1/2),    #p0+z
    T[2*MAX+2,2*MAX+2] == (q*1 + (1-q)*1/2),    #p0+x
    T[3*MAX+1,3*MAX+1] == (q*1/2 + (1-q)*1/2),  #p0-z
    T[3*MAX+2,3*MAX+2] == (q*0 + (1-q)*1/2)])    #p0-x


#Eve Guessing probabilitie on bob 
#The key is generetated with state 0 and 1 send 

PGuessB = (T[0,0]+T[MAX,MAX]+2*T[1,3]-T[1,1]-T[3,3]+2*T[MAX+1,MAX+3]-T[MAX+1,MAX+1]-T[MAX+3,MAX+3])/2

#Eve Guessing probabilitie on Alice

PGuessA = (T[3,3]+T[MAX,MAX]-T[MAX+3,MAX+3])/2

W = (2*T[1,1]-2*T[MAX+1,MAX+1]+2*T[2*MAX+2,2*MAX+2]-2*T[3*MAX+2,3*MAX+2])/2    #witness
#constraints.append(W == 2*q)

#Solving problem

#bound_value = []
#fxl = []
guessBla = []
guessAla = []
guessBlaM = []
guessAlaM = []
HBEla = []
HAEla = []
HBAla = []
KeyRa = []
KeyDa = []

RANGE = [x*0.1 for x in range(10*0, 8*1)][0:-1] + [x*0.01 for x in range(7*10, 10*10)]

print("Range =", RANGE)

for val in (RANGE):
    print("q =", val)
    probBa = cp.Problem(cp.Maximize(cp.real(PGuessB)),
                      constraints + constraints2)   
    probAa = cp.Problem(cp.Maximize(cp.real(PGuessA)),
                      constraints + constraints2) 
    probBaM = cp.Problem(cp.Maximize(cp.real(PGuessB)),
                      constraints + [W == 2*q])   
    probAaM = cp.Problem(cp.Maximize(cp.real(PGuessA)),
                      constraints + [W == 2*q])
    
    q.value = val
    print(val)
    probBa.solve(solver=cp.MOSEK)
    probAa.solve(solver=cp.MOSEK)
    probBaM.solve(solver=cp.MOSEK)
    probAaM.solve(solver=cp.MOSEK)
    #bound = BOUND(val)
    guessB = probBa.value
    guessA = probAa.value
    #bound_value.append(bound)
    #fxl.append(fx(val))
    guessBla.append(guessB)
    guessAla.append(guessA)
    guessBlaM.append(probBaM.value)
    guessAlaM.append(probAaM.value)
    
    print("optimal value with MOSEK guessB =", probBa.value, "status", probBa.status)
    print("optimal value with MOSEK guessA =", probAa.value, "status", probAa.status)
    print("optimal value with MOSEK guessBMermin =", probBaM.value, "status", probBaM.status)
    print("optimal value with MOSEK guessAMermin =", probAaM.value, "status", probAaM.status)
    
    HBA = -((1+val)*math.log2((1+val)/2) + (1-val)*math.log2((1-val)/2))/2
    InfHPguessBa = -math.log2(probBa.value)
    InfHPguessAa = -math.log2(probAa.value)
    print("H(B|A)=" ,HBA)
    print("H(B|E)>=",InfHPguessBa)
    KeyRateReversa = InfHPguessBa - HBA
    KeyRateDirecta = InfHPguessAa - HBA
    if KeyRateReversa < 0:
        KeyRa.append(0)
    elif KeyRateReversa > 0:
        KeyRa.append(KeyRateReversa)
    if KeyRateDirecta < 0:
        KeyDa.append(0)
    elif KeyRateDirecta > 0:
        KeyDa.append(KeyRateDirecta)
    
    print("Key Rate Revers", KeyRateReversa)
    print("Key Rate Direct", KeyRateDirecta)
    print("**********************************")
    
print("DONE")